In [ ]:
%%capture
%load_ext autoreload
%autoreload 2

%pip install -r requirements.txt

In [1]:
import pandas as pd

## Vuelos

In [2]:
flights = pd.read_parquet("../resources/flights.parquet")
flights.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0


In [3]:
# Registros nulos para cada campo
flights.isna().sum()

Flight_ID                0
Aeronave             80390
DepartureStation         1
ArrivalStation           1
Destination_Type         1
Origin_Type              1
STD                      0
STA                      0
Capacity                 0
Passengers          123525
Bookings            123525
dtype: int64

Se eliminan todos los casos donde `Aeropuerto` es nulo pues indica que el vuelo fue cancelado o cualquier otra situación que no es necesario considerar. Asimismo se eliminan 2 registro único donde los campos `DepartureStation`, `ArrivalStation`, `Destination_Type`, `Origin_Type` son nulos.

In [4]:
flights_ = flights.dropna(subset=["Aeronave", "Passengers", "Bookings"])
flights_ = flights_.dropna(subset=["DepartureStation", "ArrivalStation"])
flights_.isna().sum()

Flight_ID           0
Aeronave            0
DepartureStation    0
ArrivalStation      0
Destination_Type    0
Origin_Type         0
STD                 0
STA                 0
Capacity            0
Passengers          0
Bookings            0
dtype: int64

Se crea una columna de `Semana` y `Mes` para análisis posteriores.

In [11]:
flights_["STD"] = pd.to_datetime(flights_["STD"])
flights_["STA"] = pd.to_datetime(flights_["STA"])
flights_["Semana"] = flights_["STD"].dt.isocalendar().week
flights_["DiaSemana"] = flights_["STD"].dt.weekday + 1
flights_["Mes"] = flights_["STD"].dt.month
flights_

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Semana,Mes,DiaSemana
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,42,10,4
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,27,7,1
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,26,6,1
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,6,2,5
5,3b5df8805161ea827d2f2e4298c38e06,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183.0,125.0,36,9,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245718,d9ef769565064b1ff1ad90d6d18f69cb,XA-VYF,AO,BT,Playa,Ciudad Principal,2023-12-26 06:20:00,2023-12-26 10:10:00,180,166.0,89.0,52,12,2
245745,598035669a4f0fa2ce1207977932df9e,XA-VAI,AW,BT,Playa,Ciudad Principal,2023-12-28 10:25:00,2023-12-28 13:50:00,180,142.0,54.0,52,12,4
245747,90d2cab80b7700b5d854ae393678144b,XA-VYA,BA,BT,Playa,Ciudad Principal,2023-12-23 11:45:00,2023-12-23 15:00:00,180,156.0,61.0,51,12,6
245748,cc7c1c5e6fd132fd0bdab3a35aac33c0,XA-VBK,BM,BT,Playa,Ciudad Fronteriza,2023-12-29 07:30:00,2023-12-29 14:50:00,240,189.0,86.0,52,12,5


In [12]:
flights_.to_parquet("../resources/flights_clean.parquet", index=False)

## Ventas

In [ ]:
sales = pd.read_parquet("../resources/sales.parquet")

In [ ]:
sales.head()

,Flight_ID,ProductType,ProductName,Quantity,TotalSales
0,a05290288259526edd3601160b10e1de,Botanas,Carne Seca Habanero,1,55.0
1,08f6f97437df8db101b050f1110be656,Licores,Jw Red Label,2,240.0
2,4e09c949826a77207868412baeff6d30,Licores,Jack And Coke,8,576.0
3,c3e1568fe46c68d3174681d322d412b3,Licores,Jw Red Label,2,240.0
4,7396e43fb94caca87add7bb1f1611e7e,Vivabus,Vivabus_Gdl.-Nvo. C. Camionera,4,320.0


In [ ]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2564361 entries, 0 to 2564360
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Flight_ID    object 
 1   ProductType  object 
 2   ProductName  object 
 3   Quantity     int64  
 4   TotalSales   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 97.8+ MB


In [ ]:
sales.describe()

,Quantity,TotalSales
count,2.564361e+06,2.564361e+06
mean,2.519319e+00,1.110370e+02
std,2.829032e+00,1.736604e+02
min,1.000000e+00,-3.234000e+03
25%,1.000000e+00,4.500000e+01
50%,2.000000e+00,8.000000e+01
75%,3.000000e+00,1.440000e+02
max,8.500000e+01,1.290000e+04


In [ ]:
sales["Flight_ID"].value_counts().head(10)

Flight_ID
09a295c868301fe15df13d8411cd4d37    73
4c65ba84403c407f59258b9ad414e40d    73
a44008629d97a1b4c9b6e0980ae8b11a    73
edbdab3c1c0df3d2b2213343fb9144eb    73
b6a39758032c87d2e315889e36b97277    72
14069ac4ad5e26456a8f103484b84774    71
ab6dc691f1869a7c60dfd306ea408cdf    71
fd0e971f1c0af6e01be0cec64326d09b    70
c09524fd81d49de6643b61b31bae4864    69
722a5e63351b62bc5e95488f77f1c84e    69
Name: count, dtype: int64

In [ ]:
# Se eliminan valores nulos pero
# realmente no hay
sales.dropna(inplace=True)

In [ ]:
sales["ProductType"].value_counts()

ProductType
Botanas                 613793
Refrescos               585118
Specials                395760
Licores                 304987
Perecederos             181990
Galletas                143928
Bebidas Calientes       125235
Sopas                   115486
OFERTAS                  38384
Lacteos                  37755
Vivabus                   9819
Transportaciones CUN      3635
Hertz.                    2398
COMBOS CREW               2017
VIVA Taxis                1384
Transportaciones MTY      1220
VivaTransfer               435
Antros                     388
VIVA PLAY                  386
Transportaciones TLC       239
Alimentos Charter            4
Name: count, dtype: int64

Se eliminan aquellos tipos de producto que no son de interés para el análisis pues no son tangibles

In [ ]:
excluded_types = [
    "Specials",
    "OFERTAS ",
    "Vivabus",
    "Transportaciones CUN",
    "Hertz.",
    "COMBOS CREW",
    "VIVA Taxis",
    "Transportaciones MTY",
    "VivaTransfer",
    "Antros",
    "VIVA PLAY",
    "Transportaciones TLC",
    "Alimentos Charter",
]

sales = sales[~sales.ProductType.isin(excluded_types)]
sales["ProductType"].value_counts()

ProductType
Botanas              613793
Refrescos            585118
Licores              304987
Perecederos          181990
Galletas             143928
Bebidas Calientes    125235
Sopas                115486
Lacteos               37755
Name: count, dtype: int64

In [ ]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2108292 entries, 0 to 2564360
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Flight_ID    object 
 1   ProductType  object 
 2   ProductName  object 
 3   Quantity     int64  
 4   TotalSales   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 96.5+ MB


Se eliminan aquellas ventas que sean negaivas pues implican descuentos

In [ ]:
sales = sales[sales.TotalSales > 0]

In [ ]:
sales.drop(columns=["TotalSales"], inplace=True)

In [ ]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2107861 entries, 0 to 2564360
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   Flight_ID    object
 1   ProductType  object
 2   ProductName  object
 3   Quantity     int64 
dtypes: int64(1), object(3)
memory usage: 80.4+ MB


In [ ]:
sales.reset_index(inplace=True)

In [ ]:
# Usar nuevos datos de sales
sales.to_parquet("../resources/sales_cleaned.parquet")

c:\Users\Franco Mendoza\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
